## Deneme

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import DatasetDict, Dataset
import re

In [3]:
prefix = "data/tr_wiki67/trwiki-67."
file_paths = [ f"{prefix}{postfix}.raw" for postfix in ["test", "train", "val"] ]
file_paths

['data/tr_wiki67/trwiki-67.test.raw',
 'data/tr_wiki67/trwiki-67.train.raw',
 'data/tr_wiki67/trwiki-67.val.raw']

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
example_sentence = "Türkiye-İran55 ilişkileri.55_55 - 987@"
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(example_sentence)
words

['Türkiye', 'İran55', 'ilişkileri', '55_55', '987']

## Test for prepare.py script (get_raw_dataset functionality)

In [4]:
import data.tr_wiki67.prepare as prepare

prepare.get_raw_dataset()

raw dataset is not already exist, it will be created...


Saving the dataset (0/2 shards):   0%|          | 0/393794 [00:00<?, ? examples/s]

Dataset({
    features: ['examples', 'titles', 'length'],
    num_rows: 393794
})

In [5]:
from datasets import load_from_disk

ds = load_from_disk("data/tr_wiki67/raw_dataset")

In [7]:
print(ds["titles"][21])
print(ds["examples"][21])
print(ds["length"][21])

== Yolören, Zara ==
Yolören, Sivas ilinin Zara ilçesine bağlı bir köydür.
Sivas iline 92 km, Zara ilçesine 22 km uzaklıktadır
105


In [4]:
print(ds["titles"][21])
print(ds["examples"][21])
print(ds["length"][21])

== Yolören, Zara ==
Yolören, Sivas ilinin Zara ilçesine bağlı bir köydür.
Sivas iline 92 km, Zara ilçesine 22 km uzaklıktadır
105


## Build Train and Save Tokenizer

In [8]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFKC(),
     normalizers.Lowercase()]
)

# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.WhitespaceSplit(), 
                                                   pre_tokenizers.Digits(individual_digits=True),
                                                   pre_tokenizers.Punctuation()])


special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

trainer = trainers.WordPieceTrainer(vocab_size=25_000, special_tokens=special_tokens, min_frequency=5,
                                    continuing_subword_prefix="##", limit_alphabet=500)





In [19]:
print(tokenizer.normalizer.normalize_str("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?"))


?   merhabalar    555555 benim8 adım! ! .muhammet can gümüşsuüğği̇i̇çç 55  ة الزلزلة ?? ?


In [20]:
tokenizer.pre_tokenizer.pre_tokenize_str("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?")

[('?', (0, 1)),
 ('Merhabalar', (4, 14)),
 ('5', (18, 19)),
 ('5', (19, 20)),
 ('5', (20, 21)),
 ('5', (21, 22)),
 ('5', (22, 23)),
 ('5', (23, 24)),
 ('benim', (25, 30)),
 ('8', (30, 31)),
 ('adım', (32, 36)),
 ('!', (36, 37)),
 ('!', (38, 39)),
 ('.', (40, 41)),
 ('Muhammet', (41, 49)),
 ('Can', (50, 53)),
 ('GümüşsuÜğĞİİÇç', (54, 68)),
 ('5', (69, 70)),
 ('5', (70, 71)),
 ('ة', (73, 74)),
 ('الزلزلة', (75, 82)),
 ('?', (83, 84)),
 ('?', (84, 85)),
 ('?', (86, 87))]

In [15]:
from data.tr_wiki67.prepare import get_raw_dataset

dataset = get_raw_dataset()

def get_example_batch():
    """
    generator func
    will be used when training tokenizer
    """
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["examples"]

In [16]:
tokenizer.train_from_iterator(get_example_batch(), trainer=trainer)

In [21]:
encoding = tokenizer.encode("?   Merhabalar    555555 benim8 adım! ! .Muhammet Can GümüşsuÜğĞİİÇç 55  ة الزلزلة ?? ?")
print(encoding.tokens)

['?', 'mer', '##ha', '##bal', '##ar', '5', '5', '5', '5', '5', '5', 'benim', '8', 'adım', '!', '!', '.', 'muham', '##met', 'can', 'gümüş', '##su', '##üğ', '##ği', '##̇', '##i̇', '##ç', '##ç', '5', '5', 'ة', 'ال', '##ز', '##ل', '##ز', '##ل', '##ة', '?', '?', '?']


In [22]:
print(encoding)
print(encoding.ids)
print(encoding.type_ids)
print(encoding.tokens)
print(encoding.offsets)
print(encoding.attention_mask)
print(encoding.special_tokens_mask)
print(encoding.overflowing)

Encoding(num_tokens=40, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[33, 2190, 1087, 7264, 941, 25, 25, 25, 25, 25, 25, 4269, 28, 5038, 5, 5, 18, 3981, 1416, 2051, 4222, 3240, 13252, 1046, 549, 5982, 560, 560, 25, 25, 316, 8854, 605, 592, 605, 592, 596, 33, 33, 33]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['?', 'mer', '##ha', '##bal', '##ar', '5', '5', '5', '5', '5', '5', 'benim', '8', 'adım', '!', '!', '.', 'muham', '##met', 'can', 'gümüş', '##su', '##üğ', '##ği', '##̇', '##i̇', '##ç', '##ç', '5', '5', 'ة', 'ال', '##ز', '##ل', '##ز', '##ل', '##ة', '?', '?', '?']
[(0, 1), (4, 7), (7, 9), (9, 12), (12, 14), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (25, 30), (30, 31), (32, 36), (36, 37), (38, 39), (40, 41), (41, 46), (46, 49), (50, 53), (54, 59), (59, 61), (61, 63), (63, 65), (64, 65), (65, 66), (66, 67), (67, 68), (69, 70), (70, 71), (73, 

In [23]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]") 

print(cls_token_id, sep_token_id)


tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)]
)

tokenizer.decoder = decoders.WordPiece(prefix="##")

2 3


In [24]:
encoding = tokenizer.encode(" merhaba benim adım name..." , " AS gardaşım peki benim adım ne...")
print(encoding)
print(encoding.ids)
print(encoding.type_ids)
print(encoding.tokens)
print(encoding.offsets)
print(encoding.attention_mask)
print(encoding.special_tokens_mask)
print(encoding.overflowing)

Encoding(num_tokens=23, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[2, 2190, 1087, 1646, 4269, 5038, 24069, 18, 18, 18, 3, 1223, 12969, 8048, 529, 15588, 4269, 5038, 1129, 18, 18, 18, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['[CLS]', 'mer', '##ha', '##ba', 'benim', 'adım', 'name', '.', '.', '.', '[SEP]', 'as', 'gard', '##aşı', '##m', 'peki', 'benim', 'adım', 'ne', '.', '.', '.', '[SEP]']
[(0, 0), (1, 4), (4, 6), (6, 8), (9, 14), (15, 19), (20, 24), (24, 25), (25, 26), (26, 27), (0, 0), (1, 3), (4, 8), (8, 11), (11, 12), (13, 17), (18, 23), (24, 28), (29, 31), (31, 32), (32, 33), (33, 34), (0, 0)]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[]


In [85]:
tokenizer.decode(encoding.ids)

'merhaba benim adım name... as gardaşım peki benim adım ne...'

In [27]:
with open("data/tr_wiki67/trwiki-67.test.raw", "+r") as f:
    content = f.read()
    total_num_char = len(content)
    acc = len(tokenizer.encode(content).tokens)
    print(total_num_char / acc)

4.449533407710257


In [28]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer/tokenizer_55.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [30]:
wrapped_tokenizer.is_fast

True

In [32]:
wrapped_tokenizer.save_pretrained("fast_wordpiece")

('fast_wordpiece\\tokenizer_config.json',
 'fast_wordpiece\\special_tokens_map.json',
 'fast_wordpiece\\tokenizer.json')

In [38]:
test = """4 Haziran 1980'de İzmir'de doğdu. Melih Özakat İlkokulu'na, Özel Çakabey Lisesi'nde ortaokula ve konservatuvarda liseye gitti. 7 yaşında okul hayatına başlamasıyla TRT ve İzmir Devlet Konservatuvarı'nın çocuk korolarında müzik eğitimine başlaması aynı zamana denk gelir. İlk geri vokal deneyimini 12 yaşında Sezen Aksu'yla beraber 1992 yılında gerçekleştirdi. Sezen Aksu'nun yanında solo ve vokal çalışmaları devam ederken okul hayatı İzmir'de devam etmekteydi.
İlk TV programına yine Sezen Aksu'yla 1992 yılında TRT’de çıktı. Geri vokal çalışmalarının yanı sıra bu programda solo bir şarkı da seslendirmesi kamuoyunda büyük ilgi gördü. Büyük yetenek olarak lanse edilen Tuğba Özerk'in sahne ve müzik hayatı tam anlamıyla böyle başladı denilebilir. Okul hayatı bittikten sonra İstanbul’a taşınan sanatçı aralarında Ege, Deniz Seki gibi isimler bulunan sanatçılara geri vokal yaptı. Daha sonra solo sahne çalışmalarına başlayan Özerk, 2002 yılının Aralık ayında "Dün Gibi" adlı ilk solo albümünü çıkardı. 'Aşk yarası' adlı şarkısıyla uzun süre radyolarda liste başlarında yer aldı. Albümle beraber oyunculuk yönünü gösterebilme fırsatı yakalayan sanatçı, atv'nin uzun süredir yayın yapmış olduğu dizisi "Böyle mi olacaktı"da bir sene kadar rol aldı. İlk albümünde bir tane bestesi olan Özerk üç senelik boşlukta birçok besteye de imza attı.6 Temmuz 2005 tarihinde 'Lo Lo Lo' adlı ikinci solo albümünü çıkardı.'Lo Lo Lo' adlı şarkısıyla uzun süre radyolarda liste başı oldu. İkinci albümünde bir tane bestesi olan iki senelik boşlukta birçok besteye de imza attı
"""
encoding = wrapped_tokenizer(test)
print(len(encoding.tokens()))   # our tokenizer does not have any limit-boundry about block_size
encoding.tokens()[:254]

321


['[CLS]',
 '4',
 'haziran',
 '1',
 '9',
 '8',
 '0',
 "'",
 'de',
 'i̇zmir',
 "'",
 'de',
 'doğdu',
 '.',
 'meli',
 '##h',
 'öz',
 '##akat',
 'i̇lkokulu',
 "'",
 'na',
 ',',
 'özel',
 'çak',
 '##abe',
 '##y',
 'lisesi',
 "'",
 'nde',
 'ortaokul',
 '##a',
 've',
 'konservatuvar',
 '##da',
 'lise',
 '##ye',
 'gitti',
 '.',
 '7',
 'yaşında',
 'okul',
 'hayatına',
 'başlamasıyla',
 'trt',
 've',
 'i̇zmir',
 'devlet',
 'konservatuvarı',
 "'",
 'nın',
 'çocuk',
 'koro',
 '##larında',
 'müzik',
 'eğitimine',
 'başlaması',
 'aynı',
 'zamana',
 'denk',
 'gelir',
 '.',
 'i̇lk',
 'geri',
 'vokal',
 'deneyimini',
 '1',
 '2',
 'yaşında',
 'sezen',
 'aksu',
 "'",
 'yla',
 'beraber',
 '1',
 '9',
 '9',
 '2',
 'yılında',
 'gerçekleştirdi',
 '.',
 'sezen',
 'aksu',
 "'",
 'nun',
 'yanında',
 'solo',
 've',
 'vokal',
 'çalışmaları',
 'devam',
 'ederken',
 'okul',
 'hayatı',
 'i̇zmir',
 "'",
 'de',
 'devam',
 'etmekteydi',
 '.',
 'i̇lk',
 'tv',
 'programına',
 'yine',
 'sezen',
 'aksu',
 "'",
 'yla',
 '1',

## Prepare Data For (NSP-MLM) Task

In [1]:
from datasets import Dataset, load_from_disk, load_dataset
from nltk.tokenize import sent_tokenize
import os

In [2]:
# nsp_mlm_dataset = { "a_b_text": [], "x": [], "y": [], "att_mask": [], "token_type": [], "isNext": [] }

isNext = None
x = []
y = []
attention_mask = []
token_type = []
token_id = []
a_b_text = []

block_size = 255
A_segment_flag = True

In [3]:
raw_table = load_from_disk("data/tr_wiki67/raw_dataset")

In [17]:
def create_pairs(raw_row):
    # split example by sentences
    list_of_sentences = sent_tokenize(raw_row["examples"])
    A = []
    B = []

    for pair_index in range(0, len(list_of_sentences)-1):
        A.append(list_of_sentences[pair_index])
        B.append(list_of_sentences[pair_index + 1])

    return {"A": A, "B": B}


def create_a_b_table(raw_table_path):
    """
    TODO: ara kayıt işlemleri yap (save as arrow), hali hazırda var ise yükle ve döndür, examples, titles, length kolonlarını sil
    """
    # load raw dataset from disk
    raw_table = load_from_disk(raw_table_path)
    # # bu satır geçici! test'i hızlı yapabilmek için
    # raw_table = Dataset.from_dict(raw_table[:5_000])
    # every example needs at least 2 sentence
    raw_table_filtered = raw_table.filter(lambda row:  len(sent_tokenize(row["examples"], language="turkish")) > 1)


    # create table that contains A, B kols (problem: row of this table has list of sentences not just one sentence for each pair)
    a_b_list_table = raw_table_filtered.map(create_pairs)


    # 
    A, B = [], []
    for A_list, B_list in zip(a_b_list_table["A"], a_b_list_table["B"]):
        A += A_list
        B += B_list
    
    a_b_table = {"A":A, "B":B}
    a_b_table = Dataset.from_dict(a_b_table)

    return a_b_table


In [18]:
raw_table_path = "data/tr_wiki67/raw_dataset"
ab_table_path = "deneme_ab_tablosu"


# if table is not already exists
if os.path.exists(ab_table_path) == False:
    print("table is not already exists, will be created and saved to disk...")
    
    # create table
    a_b_table = create_a_b_table(raw_table_path)

    # save created table
    a_b_table.save_to_disk(ab_table_path)

# ab table is already exists
else:
    print("table is already exists, will be loaded from disk...")
    # load table from disk
    a_b_table = load_from_disk(ab_table_path)



table is not already exists, will be created and saved to disk...


Saving the dataset (0/3 shards):   0%|          | 0/4119470 [00:00<?, ? examples/s]

In [19]:
a_b_table

Dataset({
    features: ['A', 'B'],
    num_rows: 4119470
})

In [20]:
temp = a_b_table[:1000]

In [21]:
for i in range(5):
    print(f"A: {temp['A'][i]}, \nB: {temp['B'][i]}\n\n")

A: Thomas Lüthi (d. 6 Eylül 1986; Oberdiessbach), İnterwetten Paddock Moto2 takımı için, Moto2 Grand Prix Dünya Şampiyonası'nda yarışan İsviçreli profesyonel bir motosiklet yol yarışçısı., 
B: 2010 yılından bu yanan Moto2 Dünya Şampiyonası'nda yarışmaktadır.


A: 2010 yılından bu yanan Moto2 Dünya Şampiyonası'nda yarışmaktadır., 
B: Daha önce 250cc Dünya Şampiyonası ve 125cc Dünya Şampiyonası'nda yarışmıştır.


A: Daha önce 250cc Dünya Şampiyonası ve 125cc Dünya Şampiyonası'nda yarışmıştır., 
B: 2005 yılında 125cc Dünya Şampiyonası'nda şampiyon olmuştur.


A: 2005 yılında 125cc Dünya Şampiyonası'nda şampiyon olmuştur., 
B: Tüm istatistikler MotoGP.com sitesinden alınmıştır.


A: Laguna Verde, Bolivya'da, Altiplano Platosu'nun güneybatısında bir göldür., 
B: Licancabur'un eteğinde bulunur.




## NSP-MLM PREP

In [1]:
from datasets import load_from_disk
import os
import random


from tokenizers import (
    pre_tokenizers,
    processors,
    Tokenizer,
)

In [5]:
# GLOBAL VARS  

raw_table_path = "data/tr_wiki67/raw_dataset"
ab_table_path = "deneme_ab_tablosu"
block_size = 255

tokenizer = Tokenizer.from_file("tokenizer_wordpiece/tokenizer.json")

cls_id = tokenizer.token_to_id("[CLS]")
sep_id = tokenizer.token_to_id("[SEP]")
pad_id = tokenizer.token_to_id("[PAD]")
mask_id = tokenizer.token_to_id("[MASK]")

# if table is not already exists
if os.path.exists(ab_table_path) == False:
    print("table is not already exists, will be created and saved to disk...")
    
    # create table
    a_b_table = create_a_b_table(raw_table_path)

    # save created table
    a_b_table.save_to_disk(ab_table_path)

# ab table is already exists
else:
    print("table is already exists, will be loaded from disk...")
    # load table from disk
    a_b_table = load_from_disk(ab_table_path)

table is already exists, will be loaded from disk...


In [184]:
def _filter_cand_words(row):
    return row["cand"] != None

def _map_cand_words(row, num_tokens):
    ab = row["A"] + row["B"]
    ab_words = tokenizer.decode(tokenizer.encode(ab, add_special_tokens=False).ids).split(" ")

    cand = None

    for word in ab_words:
        word_ids = tokenizer.encode(word).ids
        if len(word_ids) == num_tokens:
            cand = word_ids
            break 
    
    return {"cand": cand}


def get_random_word_tokens(num_tokens, chunk_size=1_000):
    # why chunk? accessing all examples (~4M) for random word is inefficient
    # accessing dataset one by one is also inefficient (memory map, disk)
    # my solution is, define some chunk, and then apply .map() method
    chunk_idx = random.randint(len(a_b_table) - chunk_size)
    chunk_ab = a_b_table[chunk_idx: chunk_idx + chunk_size]

    candidate_words_table = chunk_ab.map(lambda row: _map_cand_words(row, num_tokens), batched=True).filter(_filter_cand_words, batched=True)
    idx = random.random(len(candidate_words_table))
    return tokenizer.encode(candidate_words_table[idx])



In [176]:
def create_nsp_mlm_table(ab_row):
    isNext = random.random() > 0.5
    x = []  # input sequence
    y = []  # target sequence
    
    if isNext == False:
        rand_b = a_b_table["B"][random.randint(0, len(a_b_table))]
        ab_row["B"] = rand_b

    encoding = tokenizer.encode(ab_row["A"], ab_row["B"])
    ab_len = len(encoding.tokens) 

    if ab_len > block_size:
        return None
    
    for word in tokenizer.decode(encoding.ids, skip_special_tokens=False).split(" "):

        if word == "[CLS]":
            x.append(cls_id)
            y.append(isNext)

        elif word == "[SEP]":
            x.append(sep_id)
            y.append(pad_id)
        
        else:
            tokens = tokenizer.encode(word, add_special_tokens=False).tokens

            prob = random.random()
            if prob > 0.85:
                prob_inner = random.random()

                
                if prob_inner < 0.8:
                    # mask condition
                    x += [mask_id for token in tokens]
                    y += tokens
                
                elif prob_inner < 0.9:
                    # corrupt condition

                    trial_limit = 10
                    trial = 1

                    while(trial <= trial_limit):
                        random_word_tokens = get_random_word_tokens(num_tokens=len(tokens), trial_limit=5000)
                        if random_word_tokens != None:
                            break
                    if random_word_tokens == None:
                        print(f"cannot find word that has {len(tokens)} number of tokens, in this {trial_limit} trials...")
                        return None
                    
                    x += get_random_word_tokens(num_tokens=len(tokens))
                    y += tokens
                
                else:
                    # identity condition
                    x += tokens
                    y += tokens
                
            else:
                x += [tokenizer.token_to_id(token) for token in tokens]
                y += [pad_id for token in tokens]

        assert len(x) == len(y), "[ERROR] there is some mistake, x and y should be the same length!"

        num_pad = block_size - len(x)
        x += [pad_id for each in range(num_pad)]
        y += [pad_id for each in range(num_pad)]

        first_sep_idx = x.index("[SEP]")
        token_type_id = [ 0 if i <= first_sep_idx else 1 for i in range(len(x)) ]

        attention_mask = [ 0 if token_id == pad_id else 1 for token_id in x ]

        return {"X": x, "Y": y, "Attention_mask": attention_mask, "Token_type_id":token_type_id, "isNext": isNext}


In [142]:
tokenizer.token_to_id("[UNK]")

0

In [ ]:
a_b_table.map(create_nsp_mlm_table, batched=True)




In [160]:
tokens = tokenizer.encode("devrimleri", add_special_tokens=False).tokens
tokens

['devrim', '##leri']

In [164]:
[tokenizer.token_to_id(each) for each in tokens]

[4181, 1034]

In [128]:
example = a_b_table[989]
example

{'A': "Beş günlük Milan (İtalyanca: Cinque giornate di Milano), İtalya'da 1848 devrimleri sırasında İtalya'nın kuzeyinde bulunan Milano'da gerçekleşen olaylar için kullanılır.",
 'B': "18 Mart 1848 yılında İtalyalı isyancılar Milano'nun sokaklarına barikat kurup yarımadadaki direnişi buraya taşıdı."}

In [138]:
encoding = tokenizer.encode(example["A"], example["B"])
print(encoding.tokens)
words = tokenizer.decode(encoding.ids, skip_special_tokens=False).split(" ")
print(words)

['[CLS]', 'beş', 'günlük', 'milan', '(', 'i̇talyanca', ':', 'cin', '##que', 'gior', '##nat', '##e', 'di', 'milano', ')', ',', 'i̇talya', "'", 'da', '1', '8', '4', '8', 'devrim', '##leri', 'sırasında', 'i̇talya', "'", 'nın', 'kuzeyinde', 'bulunan', 'milano', "'", 'da', 'gerçekleşen', 'olaylar', 'için', 'kullanılır', '.', '[SEP]', '1', '8', 'mart', '1', '8', '4', '8', 'yılında', 'i̇talya', '##lı', 'isyancı', '##lar', 'milano', "'", 'nun', 'sokak', '##larına', 'bar', '##ikat', 'kurup', 'yarımada', '##daki', 'direnişi', 'buraya', 'taşıdı', '.', '[SEP]']
['[CLS]', 'beş', 'günlük', 'milan', '(', 'i̇talyanca', ':', 'cinque', 'giornate', 'di', 'milano', '),', 'i̇talya', "'", 'da', '1', '8', '4', '8', 'devrimleri', 'sırasında', 'i̇talya', "'", 'nın', 'kuzeyinde', 'bulunan', 'milano', "'", 'da', 'gerçekleşen', 'olaylar', 'için', 'kullanılır.', '[SEP]', '1', '8', 'mart', '1', '8', '4', '8', 'yılında', 'i̇talyalı', 'isyancılar', 'milano', "'", 'nun', 'sokaklarına', 'barikat', 'kurup', 'yarımadadak

In [ ]:
# + A, B tablosu oluştur (textual formatında)
# + dataset_nsp_mlm dataset objesi/tablosu oluştur
# + dataset_nsp_mlm kolonları-key'leri -> a_b_raw, x, y, att_mask, token_type, isNext
# + isNext bool değişkeni, x, y, attention_mask, token_type_id, id, a_b_raw (birleşik row) list objeleri oluştur, block_size değişkeni, A_segment_flag:bool değişkeni olsun
# + A, B tablosunda her row'a git
# + isNext olasılığı al
# + isNext True ise o rowdaki A ve B'yi al, bu iki text'i birleştir (başa [CLS], araya ve sona [SEP] flag kelimeleri gelsin) -> a_b_raw
# + isNext False ise o rowdaki A'yı al, rastgele B al (başa [CLS], araya ve sona [SEP] f lag kelimeleri gelsin) -> a_b_raw
# + birleşik row'u tokenize et, length'i > block_size ise bu birleşik row'la işimiz yok dataset_nsp_mlm'ye eklenmeyecek, bu example'dan çık (return None)
# + A_segment_flag = True değişkeni oluştur
# + row alma işleminden sonra, loop döngüsü gir (kelime sayısı kadar döngü)
# + eğer kelime [CLS] kelimesi ise x'e [CLS] tokeni koy y'ye isNext değişken değerini(1/0) {kelime vs token ayrımına dikkat}
# + eğer kelime [SEP] kelimesi ise x'e [SEP] tokeni koy y'ye [PAD] token_id değerini koy
# + kelime [CLS], [SEP] değil ise:
# + kelimeyi tokenize et
# + olasılık al, olasılık tutmaz ise (mask, corrupt, identity) kelime tokenlarını direkt x'e koy, y'ye her token için [PAD] tokeni koy
# + olasılık mask condition'u tuttu ise kelime tokenlarını al y'ye koy, x'e de [MASK] tokenlarını koy (her kelime token'ı için tabi)
# + olasılık corrupt condition'u tuttu ise kelime token sayısını al "get_random_word_tokens(num_tokens=değer)" çağır. Gelen token'ları x'e koy, kendi kelimemizin token'larını da y'ye koy
# + get_random_word_tokens false return etti ise, deneme sayı limiti aşılmış demektir (belirtilen token sayısına uygun kelime bulunamayabilir sonuçta) bu durumda gene bu row/example'dan return null ile çıkalım
# + olasılık identity condition'u tuttu ise kelime tokenlarını al x'e ve y'ye koy
# + token_type_id listesine ilk [SEP] token ile karşılaşana kadar her x, y doldurumunda her token için 1, bundan sonra hep 0 koyulacak (A_segment_flag kullanılacak)
# + döngüden çıktıktan sonra (tüm kelimeleri gezilmiş demek):
# + x ve y aynı uzunlukta mı kontrol et, block_size - x uzunluğu kadar [PAD] tokenlı list oluştur, x ve y'ye eklemlendir
# + x uzunluğu kadar 1, block_size - x uzunluğu kadar 0 değerli list oluştur -> attention_mask liste bu list'eyi refere edecek
# + elimizdeki değişkenler ile (x, y, a_b_raw, attention mask, token type id, isNext) dataset_nsp_mlm dataset'ine yeni bir row döndüreceğiz
# + A, B tablosunda tüm rowlara girdikten sonra (.map ile), dataset_nsp_mlm tablosu tamamlanmış demektir. bu tabloyu disk'e kaydet




# ---- A, B Tablosu Oluşturma ----

# yeni bir dataset objesi oluştur
# klasik dataset objesini alttaki functionalite ile .map'le (her bir row'a git bu fonk uygula, fonk ne return ediyorsa)
# pair kerneli oluştur
# example metini (doc) sentence'lara split et
# döngü başlat (0, kaç tane sentence var ise - 1)
# pair kerneli doldur (2 sentence ile)
# yeni dataset'e extend'le, ekle 


# ---- get_random_word_tokens(num_tokens=değer) oluşturma ----#

# deneme sayısı değişkeni = 5000 (max kelime bakma sayısı)
# deneme döngüsüne gir (0'dan deneme sayısı değişkeni)
# + random_row sayı/index al [0, A,B tablosu row sayısı] : range
# + random_word sayı/index al [0, o row'da kaç tane kelime var ise] : range
# + bu index'ler ile kelimeyi al
# + kelimeyi tokenize et
# + token sayısı == num_tokens ise list objesini (içinde tokenlar var) return et
# + döngüden çıkıldı ise deneme sayısı aşılmış demektir uygun kelime bulunamamış demektir -> return null 


# UYARI: DATASET objesi ya da dataloader'da random index/sampling yapmalısın, A, B pairları hep ardışık şekilde!!!